In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from datetime import datetime, date, timedelta
df_train = pd.read_csv("../dataset/train.csv")

df_train.head()

,Unnamed: 0,sku,pack,size (GM),brand,price,POS_exposed w-1,volume_on_promo w-1,sales w-1,scope,target
0,WE 10 December 2016,2689,SINGLE,395.41,BRAND1,1.16,NaN,NaN,NaN,0,24175.0
1,WE 17 December 2016,2689,SINGLE,395.41,BRAND1,1.15,1.0,17.676112,24175.0,0,23521.0
2,WE 24 December 2016,2689,SINGLE,395.41,BRAND1,1.16,1.0,24.482803,23521.0,0,22075.0
3,WE 31 December 2016,2689,SINGLE,395.41,BRAND1,1.16,0.0,19.410646,22075.0,0,16492.0
4,WE 07 January 2017,2689,SINGLE,395.41,BRAND1,1.16,0.0,29.812030,16492.0,0,25971.0


In [2]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [3]:
date_list = [[x.split(" ")[1],x.split(" ")[2], x.split(" ")[3]] for x in df_train["Unnamed: 0"]]

In [4]:
df_train['day'] = [int(x[0]) for x in date_list]
df_train['month'] = [int(datetime.strptime(x[1], "%B").month) for x in date_list]
df_train['year'] = [int(x[2]) for x in date_list]

In [5]:
df_train = df_train.drop(['Unnamed: 0'], axis=1)

In [6]:
df_train.head()

,sku,pack,size (GM),brand,price,POS_exposed w-1,volume_on_promo w-1,sales w-1,scope,target,day,month,year
0,2689,SINGLE,395.41,BRAND1,1.16,NaN,NaN,NaN,0,24175.0,10,12,2016
1,2689,SINGLE,395.41,BRAND1,1.15,1.0,17.676112,24175.0,0,23521.0,17,12,2016
2,2689,SINGLE,395.41,BRAND1,1.16,1.0,24.482803,23521.0,0,22075.0,24,12,2016
3,2689,SINGLE,395.41,BRAND1,1.16,0.0,19.410646,22075.0,0,16492.0,31,12,2016
4,2689,SINGLE,395.41,BRAND1,1.16,0.0,29.812030,16492.0,0,25971.0,7,1,2017


In [7]:
df_train = df_train.fillna(0)

In [8]:
df_train = df_train.sort_values(by=['year','month','day']).reset_index(drop=True)

In [9]:
df_train.head()

,sku,pack,size (GM),brand,price,POS_exposed w-1,volume_on_promo w-1,sales w-1,scope,target,day,month,year
0,2689,SINGLE,395.41,BRAND1,1.16,0.0,0.0,0.0,0,24175.0,10,12,2016
1,1027,SINGLE,114.23,BRAND2,0.52,0.0,0.0,0.0,1,54158.0,10,12,2016
2,2696,SINGLE,395.41,BRAND1,1.14,0.0,0.0,0.0,0,14100.0,10,12,2016
3,2698,SINGLE,395.41,BRAND1,1.13,0.0,0.0,0.0,0,5591.0,10,12,2016
4,1035,SINGLE,114.23,BRAND2,0.51,0.0,0.0,0.0,1,41870.0,10,12,2016


In [10]:
# transf to categorical
df_train["brand"] = df_train["brand"].astype('category')
df_train["brand"] = df_train["brand"].cat.codes
df_train["pack"] = df_train["pack"].astype('category')
df_train["pack"] = df_train["pack"].cat.codes

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train.drop(['target','scope'],axis=1), df_train.target, test_size=0.2, shuffle=False)

model = lgb.LGBMRegressor()

model.fit(X_train, y_train,  categorical_feature=['pack', 'brand','day','year','month'])

y_pred = model.predict(X_test)

mean_absolute_percentage_error(y_test, y_pred) # 9.934343915710013 con tutte le features

c:\users\blueace17\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['brand', 'day', 'month', 'pack', 'year']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


16.49684131218611

# Natale

In [12]:
days_to_christmas = []
for d,m,y in zip(df_train.day, df_train.month, df_train.year):
    days_to_christmas.append((date(y,12,25)-date(y,m,d)).days)

In [13]:
df_train['days_to_christmas'] = days_to_christmas

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train.drop(['target','scope'],axis=1), df_train.target, test_size=0.2, shuffle=False)

model = lgb.LGBMRegressor()

model.fit(X_train, y_train,  categorical_feature=['pack', 'brand','day','year','month'])

y_pred = model.predict(X_test)

mean_absolute_percentage_error(y_test, y_pred)

13.787627855216572

In [25]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X_train, y_train)

preds = reg.predict(X_test)

mean_absolute_percentage_error(preds, y_test)

208.00991069972318